# Comprehensive Healthcare Data Analysis Using SQL

**By:** Siddhesh Mishra

**Date:** July 17, 2024

**Introduction:**
In the modern healthcare landscape, data analytics plays a crucial role in enhancing patient care, optimizing resource management, and ensuring financial sustainability. This project leverages SQL to conduct an in-depth analysis of healthcare data, providing valuable insights that can drive informed decision-making and improve hospital operations.

**The Backstory:**
Data analytics offers transformative benefits for the healthcare industry, including:

1. **Proactive Health Management:** By analyzing patient data, hospitals can identify at-risk patients early and implement preventive measures. This proactive approach can reduce the incidence of severe health issues and improve overall patient health outcomes.
2. **Operational Efficiency:** Data analytics helps streamline hospital operations by optimizing staffing, reducing wait times, and improving patient flow. Efficient resource management leads to better patient care and reduced operational costs.
3. **Personalized Medical Care:** Utilizing data to understand individual patient needs and responses to treatments enables hospitals to provide personalized medical care. This tailored approach enhances treatment effectiveness and patient satisfaction.

Balancing cost control with delivering high-quality patient care is a persistent challenge for hospitals. This requires a data-centric approach focused on continuous improvement. Effective cost management is essential for the hospital's financial health. Key metrics such as Facility Utilization—encompassing occupancy rates, patient stay durations, and bed utilization—are crucial for reducing overhead costs and increasing revenue.

**The Scenario:**
As a Data Scientist at a hospital, you have been tasked by the C-Suite to prepare a detailed report addressing the following questions:
- What is the typical length of stay for patients in the hospital?
- Is there a correlation between the number of lab procedures and the duration of hospital stay?
- What are the different racial groups present in the dataset, and how are they distributed in terms of lab procedures?
- What are the gender distributions in the dataset?
- Which medical specialties have the highest average number of procedures?
- What is the readmission rate for different medical specialties?
- Which age groups have the highest average number of procedures and medications?

**The Data:**
The dataset for this project, sourced from a Kaggle challenge on predicting hospital readmission rates for diabetic patients, includes clinical care data from 130 U.S. hospitals over a ten-year period (1998 - 2008). It comprises two tables: one with demographic information and the other with health and clinical care data.

**Purpose and Approach:**
This project aims to practice advanced SQL techniques to extract and analyze meaningful insights from the dataset. Techniques used in this project include:
- **JOINS** to combine data from multiple tables.
- **CASE statements** to create conditional logic within queries.
- **HAVING clauses** to filter groups of rows.
- **Subqueries** to perform operations on the results of other queries.
- **Common Table Expressions (CTEs)** to simplify complex queries.
- **Correlated Subqueries** to refer to columns from the outer query.
- **Aggregate Functions** to calculate averages, counts, and sums.
The goal is to apply these skills to real-world data, providing actionable recommendations to improve hospital performance.


In [1]:
import pandas as pd
import sqlite3

# Load your CSV files into pandas DataFrames
file1 = r'C:\Users\HP\OneDrive\Desktop\healthcare_sql\health.csv'
file2 = r'C:\Users\HP\OneDrive\Desktop\healthcare_sql\demographics.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('healthcare_project.db')

# Write the DataFrames to the SQLite database
df1.to_sql('health', conn, if_exists='replace', index=False)
df2.to_sql('demographics', conn, if_exists='replace', index=False)

# Verify the tables were created correctly
query1 = pd.read_sql_query("SELECT * FROM health;", conn)
query2 = pd.read_sql_query("SELECT * FROM demographics;", conn)

print("Health Table:")
print(query1)

print("\n Demographics Table:")
print(query2)



Health Table:
        encounter_id  patient_nbr  admission_type_id  \
0            2278392      8222157                  6   
1             149190     55629189                  1   
2              64410     86047875                  1   
3             500364     82442376                  1   
4              16680     42519267                  1   
...              ...          ...                ...   
101761     443847548    100162476                  1   
101762     443847782     74694222                  1   
101763     443854148     41088789                  1   
101764     443857166     31693671                  2   
101765     443867222    175429310                  1   

        discharge_disposition_id  admission_source_id  time_in_hospital  \
0                             25                    1                 1   
1                              1                    7                 3   
2                              1                    7                 2   
3            

#### Analysis 1: Total Number of Records in the Dataset
This analysis calculates the total number of records in the `health` table. 


In [2]:
query3 = "SELECT COUNT(*) AS count FROM health;"
df_count_health = pd.read_sql_query(query3, conn)
print("Number of records in the dataset:")
print(df_count_health)

Number of records in the dataset:
    count
0  101766


Setting '?' values in the dataset to NULL values so they can be used in the analysis.

In [3]:
cursor = conn.cursor()
update_queries = [
    "UPDATE demographics SET race = NULL WHERE race = '?';",
    "UPDATE demographics SET gender = NULL WHERE gender = '?';",
    "UPDATE health SET medical_specialty = NULL WHERE medical_specialty = '?';",
    "UPDATE health SET payer_code = NULL WHERE payer_code = '?';"
]

for query in update_queries:
    cursor.execute(query)
    
conn.commit()

#### Analysis 2: Distinct Values in Demographics and Medical Specialties
This analysis retrieves the distinct values for race, gender, and medical specialty present in the dataset. By identifying the unique categories within these fields, we can better understand the diversity of the patient population and the variety of medical specialties covered in the data.

In [4]:
query4 = "SELECT DISTINCT race FROM demographics WHERE race IS NOT NULL;"
query5 = "SELECT DISTINCT gender FROM demographics;"
query6 = "SELECT DISTINCT medical_specialty FROM health WHERE medical_specialty IS NOT NULL;"
df_distinct_race = pd.read_sql_query(query4, conn)
df_distinct_gender = pd.read_sql_query(query5, conn)
df_distinct_specialty = pd.read_sql_query(query6, conn)
print("List of races present in the data:")
print(df_distinct_race)
print("\nList of genders present in the data:")
print(df_distinct_gender)
print("\nList of medical specialties present in the data:")
print(df_distinct_specialty)

List of races present in the data:
              race
0        Caucasian
1  AfricanAmerican
2            Other
3            Asian
4         Hispanic

List of genders present in the data:
            gender
0           Female
1             Male
2  Unknown/Invalid

List of medical specialties present in the data:
           medical_specialty
0   Pediatrics-Endocrinology
1           InternalMedicine
2     Family/GeneralPractice
3                 Cardiology
4            Surgery-General
..                       ...
67              Perinatology
68           Neurophysiology
69  Endocrinology-Metabolism
70                   DCPTEAM
71                  Resident

[72 rows x 1 columns]


#### Analysis 3: Number of Encounters per Medical Specialty and Average Time in Hospital


In [5]:
query7 = """
SELECT medical_specialty,
COUNT(*) AS encounter_count
FROM health
WHERE medical_specialty IS NOT NULL
GROUP BY medical_specialty;"""

query8 = "SELECT AVG(time_in_hospital) AS avg_time_in_hospital FROM health;"

df_encounters_per_specialty = pd.read_sql_query(query7, conn)
df_avg_time_in_hospital = pd.read_sql_query(query8, conn)

print("Number of records for each medical specialty:")
print(df_encounters_per_specialty)
print("\nAverage time in hospital for all patients:")
print(df_avg_time_in_hospital)

Number of records for each medical specialty:
                   medical_specialty  encounter_count
0               AllergyandImmunology                7
1                     Anesthesiology               12
2           Anesthesiology-Pediatric               19
3                         Cardiology             5352
4               Cardiology-Pediatric                7
..                               ...              ...
67  Surgery-PlasticwithinHeadandNeck                1
68                  Surgery-Thoracic              109
69                  Surgery-Vascular              533
70                 SurgicalSpecialty               33
71                           Urology              685

[72 rows x 2 columns]

Average time in hospital for all patients:
   avg_time_in_hospital
0              4.395987


#### Analysis 4: Number of Encounters Categorized by Race
This analysis calculates the number of encounters categorized by race. By grouping the data by race, we can gain insights into the distribution of healthcare encounters among different racial groups.


In [30]:
query9 = """
SELECT COUNT(*) AS encounter_count,d.race
FROM health h JOIN demographics d ON h.patient_nbr = d.patient_nbr
GROUP BY d.race
ORDER BY encounter_count DESC;
"""

df_encounters_by_race = pd.read_sql_query(query9, conn)
print("Number of encounters categorized by race:")
print(df_encounters_by_race)

Number of encounters categorized by race:
   encounter_count             race
0           171053        Caucasian
1            47271  AfricanAmerican
2             4194         Hispanic
3             3337             None
4             2799            Other
5             1238            Asian



The analysis indicates that the majority of healthcare encounters involve Caucasian and African American patients, with Hispanic, Other, and Asian patients representing smaller portions of the patient population. A notable number of encounters are uncategorized, highlighting a need for improved data collection. These insights suggest that healthcare facilities should prioritize resources and interventions for the predominant groups while also addressing the needs of smaller racial groups and enhancing data accuracy to better understand and mitigate healthcare disparities.

#### Analysis 5: Average Number of Lab Procedures by Age Group
This analysis calculates the average number of lab procedures performed for each age group. By grouping the data by age, we can gain insights into how the use of lab procedures varies across different age demographics.


In [29]:
query10 = """
SELECT AVG(h.num_lab_procedures) AS avg_lab_procedures, d.age
FROM health h JOIN demographics d ON h.patient_nbr = d.patient_nbr
GROUP BY d.age;
"""

df_avg_lab_procedures_by_age = pd.read_sql_query(query10, conn)
print("Average number of lab procedures by age:")
print(df_avg_lab_procedures_by_age)

Average number of lab procedures by age:
   avg_lab_procedures       age
0           40.508475    [0-10)
1           44.776887   [10-20)
2           41.227052   [20-30)
3           43.516695   [30-40)
4           43.239461   [40-50)
5           42.884059   [50-60)
6           42.970463   [60-70)
7           43.008460   [70-80)
8           43.144321   [80-90)
9           43.337536  [90-100)


The analysis indicates that the average number of lab procedures is relatively consistent across different age groups, suggesting that diagnostic testing is uniformly distributed among patients of varying ages. This consistency implies that healthcare providers administer a similar amount of lab procedures regardless of age, which can inform resource allocation and service planning to ensure that lab services are adequately provided for all age demographics.

#### Analysis 6: Categorizing Patients by Length of Hospital Stay
This analysis categorizes patients based on their length of hospital stay into three categories: 'Short stay', 'Medium stay', and 'Long stay'. This categorization helps to understand the distribution of patient stays and identify patterns related to different lengths of hospitalization.


In [28]:
query11 = """
SELECT patient_nbr, time_in_hospital,
CASE WHEN time_in_hospital < 3 THEN 'Short stay' 
     WHEN time_in_hospital BETWEEN 3 AND 7 THEN 'Medium stay' 
     ELSE 'Long stay' END AS stay_length FROM health;
"""

df_stay_length = pd.read_sql_query(query11, conn)
print("Patient length of stay:")
print(df_stay_length)

Patient length of stay:
        patient_nbr  time_in_hospital  stay_length
0           8222157                 1   Short stay
1          55629189                 3  Medium stay
2          86047875                 2   Short stay
3          82442376                 2   Short stay
4          42519267                 1   Short stay
...             ...               ...          ...
101761    100162476                 3  Medium stay
101762     74694222                 5  Medium stay
101763     41088789                 1   Short stay
101764     31693671                10    Long stay
101765    175429310                 6  Medium stay

[101766 rows x 3 columns]


#### Analysis 7: Average Length of Stay for Readmitted Patients
This analysis calculates the average length of stay in the hospital for patients who were readmitted. By focusing on readmitted patients, we can gain insights into how long these patients typically stay in the hospital, which can help in understanding and addressing factors related to readmissions.

In [31]:
query12 = """
SELECT AVG(time_in_hospital) AS avg_stay
FROM health
WHERE patient_nbr IN 
    (SELECT patient_nbr FROM health WHERE readmitted != 'NO');
"""

df_avg_stay_readmitted = pd.read_sql_query(query12, conn)
print("Average length of stay for patients who were readmitted:")
print(df_avg_stay_readmitted)

Average length of stay for patients who were readmitted:
   avg_stay
0  4.574637


The analysis reveals that the average time in the hospital for all patients is approximately 4.4 days, while the average length of stay for patients who were readmitted is slightly higher at around 4.6 days. This indicates that patients who are readmitted tend to have longer initial hospital stays compared to the general patient population. This difference suggests that patients with more complex or severe conditions, who require longer initial stays, may be at a higher risk of readmission. Understanding this can help healthcare providers focus on improving care and discharge planning for patients with extended hospital stays to potentially reduce readmission rates.

#### Analysis 8: Average Procedures and Medications by Age Group
This analysis calculates the average number of procedures and medications for each age group. By grouping the data by age, we can gain insights into how medical interventions vary across different age groups.


In [20]:
query13 = """
SELECT d.age, 
       ROUND(AVG(h.num_procedures), 1) AS avg_procedures, 
       ROUND(AVG(h.num_medications), 1) AS avg_medications
FROM health h
JOIN demographics d ON h.patient_nbr = d.patient_nbr
GROUP BY d.age
ORDER BY d.age;
"""
df_avg_by_age = pd.read_sql_query(query13, conn)
print(df_avg_by_age)

        age  avg_procedures  avg_medications
0    [0-10)             0.2              6.1
1   [10-20)             0.3              9.1
2   [20-30)             0.6             12.6
3   [30-40)             1.0             14.8
4   [40-50)             1.2             16.3
5   [50-60)             1.4             17.4
6   [60-70)             1.4             17.7
7   [70-80)             1.2             16.7
8   [80-90)             0.9             15.7
9  [90-100)             0.7             13.6


The analysis indicates that the average number of procedures and medications both tend to increase with age, peaking in the 60-70 age range for procedures and in the 60-70 age range for medications, before declining in the older age groups. This trend suggests that middle-aged and older adults generally require more medical interventions and medications, reflecting higher healthcare needs in these age groups. The decline in the oldest age groups may be due to various factors, including different healthcare strategies or fewer aggressive treatments. 

#### Analysis 9: Top Medical Specialties by Average Number of Procedures
This analysis identifies medical specialties with the highest average number of procedures. By calculating the average number of procedures per medical specialty and including only those specialties with more than 50 encounters, we ensure the results are statistically significant. This helps in understanding which specialties tend to involve more procedures on average, potentially indicating the complexity or intensity of care provided in those areas.


In [19]:
query14 = """
SELECT medical_specialty, 
       ROUND(AVG(num_procedures),1) AS avg_procedures, 
       COUNT(*) AS count
FROM health
GROUP BY medical_specialty
HAVING COUNT(*) > 50
ORDER BY avg_procedures DESC;
"""
df_avg_procedures = pd.read_sql_query(query14, conn)
print(df_avg_procedures)

                    medical_specialty  avg_procedures  count
0                    Surgery-Thoracic             3.5    109
1     Surgery-Cardiovascular/Thoracic             3.2    652
2                         Radiologist             3.2   1140
3                          Cardiology             2.7   5352
4                    Surgery-Vascular             2.6    533
5                           Radiology             2.5     53
6                            Podiatry             2.4    100
7              Surgery-Cardiovascular             2.1     98
8                           Neurology             2.1    203
9                          Gynecology             2.1     58
10                      Surgery-Neuro             1.9    468
11                     Otolaryngology             1.9    125
12                        Orthopedics             1.9   1400
13            ObstetricsandGynecology             1.9    671
14                            Urology             1.8    685
15                      

The analysis indicates that medical specialties such as Surgery-Thoracic, Surgery-Cardiovascular/Thoracic, and Radiology have the highest average number of procedures, suggesting these specialties involve more complex or intensive treatments. In contrast, specialties like Psychiatry, Psychology, and Pediatrics-Endocrinology have the lowest average number of procedures, reflecting their focus on treatments that may not require many procedures. This variation in procedure intensity across specialties highlights the diverse nature of medical practices and can inform resource allocation, training, and strategic planning in healthcare facilities to cater to the specific needs of each specialty.

#### Analysis 10: Readmission Rate by Medical Specialty
This analysis calculates the readmission rate for each medical specialty. We consider the medical specialties where the number of encounters are greater than 50 for statistically significant results. The goal of this analysis is to identify which medical specialties have the highest readmission rates, which can help in understanding and potentially addressing issues related to patient care in those specialties.


In [26]:
query15 = """
# CTE for readmissions
WITH readmissions AS (
    SELECT medical_specialty, COUNT(*) AS readmission_count
    FROM health
    WHERE readmitted != 'NO' 
    GROUP BY medical_specialty
    HAVING COUNT(*) > 50
),
# CTE for encounters
total_encounters AS (
    SELECT medical_specialty, COUNT(*) AS total_count
    FROM health
    GROUP BY medical_specialty 
    HAVING COUNT(*) > 50
)
# Selecting data from CTEs
SELECT t.medical_specialty, r.readmission_count,
    (r.readmission_count * 1.0 / t.total_count) AS readmission_rate # 
FROM total_encounters t
LEFT JOIN readmissions r ON t.medical_specialty = r.medical_specialty
WHERE t.medical_specialty IS NOT NULL 
ORDER BY readmission_rate DESC;
"""
df_readmission_rate = pd.read_sql_query(query15, conn)
print(df_readmission_rate)

                    medical_specialty  readmission_count  readmission_rate
0                            Podiatry               58.0          0.580000
1                          Nephrology              914.0          0.566646
2                    Surgery-Vascular              274.0          0.514071
3                    Emergency/Trauma             3852.0          0.509187
4                 Hematology/Oncology              104.0          0.502415
5                    Gastroenterology              276.0          0.489362
6                         Pulmonology              424.0          0.486797
7                            Oncology              166.0          0.477011
8              Family/GeneralPractice             3546.0          0.476613
9                     Surgery-General             1388.0          0.447886
10                   InternalMedicine             6369.0          0.435190
11                         Psychiatry              366.0          0.428571
12                       

The analysis indicates that medical specialties such as Podiatry, Nephrology, and Surgery-Vascular have the highest readmission rates, suggesting these specialties may involve conditions that require more follow-up care or have higher complications. In contrast, specialties like Surgery-Neuro, Obstetrics and Gynecology, and Pediatrics-CriticalCare have lower readmission rates, potentially indicating more effective initial treatments or lower complexity of cases. Specialties with no data (NaN values) for readmission rates need further investigation to understand if it's due to insufficient data or other factors. This information can help healthcare providers identify areas needing improvement and allocate resources to reduce readmission rates effectively.

**Conclusion:**
This project demonstrates the power of advanced SQL techniques in extracting actionable insights from healthcare data. By analyzing patient demographics, medical procedures, and hospital operations, we identified key patterns and correlations that can inform strategic decision-making. The findings highlight the importance of understanding patient length of stay, procedural loads, and readmission rates, providing a foundation for targeted interventions. Adopting a data-driven approach enables hospitals to enhance patient care, optimize resource utilization, and achieve sustainable improvements in patient outcomes and operational efficiency.
